# Senko in Google Colab

Install `uv`

In [1]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

downloading uv 0.9.8 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!


Verify `uv` installation

In [2]:
!/usr/local/bin/uv --version

uv 0.9.8


Install Senko

In [3]:
!/usr/local/bin/uv pip install --system "git+https://github.com/narcotic-sh/senko.git[nvidia]"

Using Python 3.12.12 environment at: /usr
Resolved 246 packages in 21.60s
Prepared 71 packages in 1m 37s
Uninstalled 31 packages in 706ms
Installed 71 packages in 116ms
 + asteroid-filterbanks==0.4.0
 - bokeh==3.7.3
 + bokeh==3.6.3
 + coloredlogs==15.0.1
 + colorlog==6.10.1
 + colour-science==0.4.6
 + cucim-cu12==25.8.0
 + cuda-bindings==12.9.4
 + cuda-pathfinder==1.3.2
 - cuda-python==12.6.2.post1
 + cuda-python==12.9.4
 - cudf-cu12==25.6.0 (from https://pypi.nvidia.com/cudf-cu12/cudf_cu12-25.6.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl)
 + cudf-cu12==25.8.0
 + cugraph-cu12==25.8.0
 - cuml-cu12==25.6.0
 + cuml-cu12==25.8.0
 - cuvs-cu12==25.6.1
 + cuvs-cu12==25.8.0
 + cuxfilter-cu12==25.8.0
 - dask==2025.5.0
 + dask==2025.7.0
 - dask-cuda==25.6.0
 + dask-cuda==25.8.0
 - dask-cudf-cu12==25.6.0
 + dask-cudf-cu12==25.8.0
 + datashader==0.18.2
 - distributed==2025.5.0
 + distributed==2025.7.0
 - distributed-ucxx-cu12==0.44.0
 + distributed-ucxx-cu12==0.45.1
 + docopt==0.

# 구글 마운트


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [4]:
from google.colab import files

# 파일 업로드
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print(f"업로드된 파일: {audio_path}")

Saving 1106_오후회의.m4a to 1106_오후회의.m4a
업로드된 파일: 1106_오후회의.m4a


# Sampling
1. .m4a to .wav
2. match 16kHz
3. 16bit

filePath만 지정하면 된다.

In [5]:
import os
import subprocess
from pydub import AudioSegment
from pydub.effects import normalize, high_pass_filter
from pydub.silence import detect_nonsilent
import numpy as np

# 출력 디렉토리 생성
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# WAV로 변환 (원본)
wav_path = os.path.join(output_dir, "audio.wav")
# 전처리된 파일 경로
preprocessed_path = os.path.join(output_dir, "audio_preprocessed.wav")

print("🔄 오디오 변환 및 전처리 중...")

# ffmpeg로 직접 변환
subprocess.run([
    'ffmpeg', '-i', audio_path,
    '-ar', '16000',  # 16kHz
    '-ac', '1',      # mono
    '-y',            # overwrite
    wav_path
], capture_output=True, check=True)

# 변환된 파일 로드
audio = AudioSegment.from_wav(wav_path)

print(f"   원본 길이: {len(audio) / 1000:.1f}초")
print(f"   원본 볼륨: {audio.dBFS:.1f}dB")

# ===== 전처리 시작 =====

# 1. High-pass 필터 (저주파 노이즈 제거)
print("   🔧 저주파 노이즈 제거 중...")
audio = high_pass_filter(audio, cutoff=80)  # 80Hz 이하 제거

# 2. 무음 구간 최소화 (앞뒤 무음 제거)
print("   🔧 무음 구간 정리 중...")
# 무음이 아닌 구간 탐지 (threshold: -50dB, min_silence_len: 100ms)
nonsilent_ranges = detect_nonsilent(
    audio,
    min_silence_len=100,
    silence_thresh=-50
)

if nonsilent_ranges:
    # 첫 소리 시작과 마지막 소리 끝 찾기
    start_trim = max(0, nonsilent_ranges[0][0] - 500)  # 0.5초 여유
    end_trim = min(len(audio), nonsilent_ranges[-1][1] + 500)  # 0.5초 여유
    audio = audio[start_trim:end_trim]

# 3. 적응형 볼륨 증폭
print("   🔧 볼륨 증폭 중...")
current_db = audio.dBFS

if current_db < -20:  # 너무 작은 경우 (약 0.3 amplitude에 해당)
    # 목표 볼륨: -3dB (약 0.7 amplitude)
    target_db = -3
    gain_needed = target_db - current_db
    audio = audio + gain_needed
    print(f"   📈 볼륨 증폭: +{gain_needed:.1f}dB ({current_db:.1f}dB → {audio.dBFS:.1f}dB)")

elif current_db < -12:  # 약간 작은 경우 (약 0.5 amplitude에 해당)
    # 목표 볼륨: -2.5dB (약 0.75 amplitude)
    target_db = -2.5
    gain_needed = target_db - current_db
    audio = audio + gain_needed
    print(f"   📈 볼륨 증폭: +{gain_needed:.1f}dB ({current_db:.1f}dB → {audio.dBFS:.1f}dB)")

else:
    # 충분히 큰 경우: -1.5dB로 정규화 (약 0.85 amplitude)
    audio = normalize(audio, headroom=1.5)
    print(f"   ✓ 볼륨 정규화: {audio.dBFS:.1f}dB")

# 4. 소프트 클리핑 방지 (피크 제한)
# pydub의 normalize는 자동으로 클리핑을 방지하지만, 추가 안전장치
if audio.max_dBFS > -0.5:  # 피크가 너무 높으면
    audio = audio - (audio.max_dBFS + 1.0)  # -1dB로 제한
    print("   🛡️ 클리핑 방지 적용")

# 전처리된 오디오 저장
print("   💾 전처리된 오디오 저장 중...")
audio.export(preprocessed_path, format="wav")

duration = len(audio) / 1000
final_db = audio.dBFS
print(f"✅ 전처리 완료: {preprocessed_path}")
print(f"   샘플레이트: {audio.frame_rate}Hz")
print(f"   최종 길이: {duration:.1f}초")
print(f"   최종 볼륨: {final_db:.1f}dB")
print(f"   품질: 노이즈 제거 + 볼륨 최적화 완료")

🔄 오디오 변환 및 전처리 중...


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


   원본 길이: 2086.0초
   원본 볼륨: -26.7dB
   🔧 저주파 노이즈 제거 중...
   🔧 무음 구간 정리 중...
   🔧 볼륨 증폭 중...
   📈 볼륨 증폭: +24.2dB (-27.2dB → -6.5dB)
   🛡️ 클리핑 방지 적용
   💾 전처리된 오디오 저장 중...
✅ 전처리 완료: output/audio_preprocessed.wav
   샘플레이트: 16000Hz
   최종 길이: 2086.0초
   최종 볼륨: -7.5dB
   품질: 노이즈 제거 + 볼륨 최적화 완료


# 4. [모듈 B] Speaker Diarization (화자 분리)

In [6]:
import json

def convert_senko_to_custom_format(senko_result):
    """
    Senko 결과를 커스텀 JSON 형식으로 변환

    Args:
        senko_result: Senko diarizer의 결과 딕셔너리

    Returns:
        dict: 변환된 결과 딕셔너리
    """

    # 1. turns 데이터 생성 (merged_segments 활용)
    turns = []
    for segment in senko_result['merged_segments']:
        turns.append({
            "speaker_label": segment['speaker'],
            "start": round(segment['start'], 2),  # 소수점 2자리
            "end": round(segment['end'], 2)
        })

    # 2. embeddings 데이터 생성 (speaker_centroids 활용)
    embeddings = {}
    for speaker, centroid in senko_result['speaker_centroids'].items():
        # numpy array를 list로 변환
        embeddings[speaker] = centroid.tolist() if hasattr(centroid, 'tolist') else list(centroid)

    # 3. 최종 결과 구성
    result = {
        "turns": turns,
        "embeddings": embeddings
    }

    print(f"✅ 변환 완료: {len(turns)}개 구간, {len(embeddings)}명 화자")

    return result

In [7]:
from google.colab import files
import json
import senko

# 1. Senko 실행
print("🎯 화자 분리 시작...")
diarizer = senko.Diarizer(device='auto', warmup=True, quiet=False)
senko_result = diarizer.diarize(preprocessed_path, generate_colors=False)

# 2. 커스텀 포맷으로 변환
custom_result = convert_senko_to_custom_format(senko_result)

# 3. JSON 파일로 저장 및 다운로드
output_filename = "diarization_result.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(custom_result, f, ensure_ascii=False, indent=2)

print(f"📥 다운로드 시작: {output_filename}")
files.download(output_filename)

🎯 화자 분리 시작...
Using device: cuda


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover



Loading embedding model ........ done [0.6s]
Warming up embedding model ..... done [3.4s]
Using GPU clustering
Warming up clustering objects ... done [2.1s]

    audio_preprocessed.wav
      ├── Voice activity detection ..... done [2.71s]
      ├── Fbank feature extraction ..... done [4.09s]
      ├── Embeddings generation ........ done [1.24s]
      └── Clustering ................... done [0.72s]

    Total diarization time: 8.77s

✅ 변환 완료: 275개 구간, 5명 화자
📥 다운로드 시작: diarization_result.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 구버전


In [ ]:
# import json

# def convert_senko_to_custom_format(senko_result, output_path):
#     """
#     Senko 결과를 커스텀 JSON 형식으로 변환

#     Args:
#         senko_result: Senko diarizer의 결과 딕셔너리
#         output_path: 저장할 JSON 파일 경로
#     """

#     # 1. turns 데이터 생성 (merged_segments 활용)
#     turns = []
#     for segment in senko_result['merged_segments']:
#         turns.append({
#             "speaker_label": segment['speaker'],
#             "start": round(segment['start'], 2),  # 소수점 2자리
#             "end": round(segment['end'], 2)
#         })

#     # 2. embeddings 데이터 생성 (speaker_centroids 활용)
#     embeddings = {}
#     for speaker, centroid in senko_result['speaker_centroids'].items():
#         # numpy array를 list로 변환
#         embeddings[speaker] = centroid.tolist() if hasattr(centroid, 'tolist') else list(centroid)

#     # 3. 최종 결과 구성
#     result = {
#         "turns": turns,
#         "embeddings": embeddings
#     }

#     # 4. JSON 파일로 저장
#     with open(output_path, 'w', encoding='utf-8') as f:
#         json.dump(result, f, ensure_ascii=False, indent=2)

#     print(f"✅ 변환 완료: {len(turns)}개 구간, {len(embeddings)}명 화자")
#     print(f"📁 저장 위치: {output_path}")

#     return result

In [ ]:
# # Senko 실행 (노트북 코드 기준)
# import senko

# diarizer = senko.Diarizer(device='auto', warmup=True, quiet=False)
# senko_result = diarizer.diarize(preprocessed_path, generate_colors=False)

# # senko_result = result

# # 커스텀 형식으로 변환 및 저장
# output_path = "/content/drive/MyDrive/final_project/1105_회의_custom.json"
# custom_result = convert_senko_to_custom_format(senko_result, output_path)

# # 결과 미리보기
# print("\n=== Turns 샘플 (처음 3개) ===")
# for turn in custom_result['turns'][:3]:
#     print(f"{turn['speaker_label']}: {turn['start']}s ~ {turn['end']}s")

# print("\n=== Embeddings 정보 ===")
# for speaker, emb in custom_result['embeddings'].items():
#     print(f"{speaker}: 차원 {len(emb)}")

Using device: cuda


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover



Loading embedding model ........ done [0.6s]
Warming up embedding model ..... done [3.5s]
Using GPU clustering
Warming up clustering objects ... done [2.1s]

    audio_preprocessed.wav
      ├── Voice activity detection ..... done [2.70s]
      ├── Fbank feature extraction ..... done [4.10s]
      ├── Embeddings generation ........ done [1.20s]
      └── Clustering ................... done [0.71s]

    Total diarization time: 8.72s



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/final_project/1105_회의_custom.json'

In [ ]:
custom_result

# [모듈 A] STT (Whisper)

In [8]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-zscm9ayz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-zscm9ayz
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=fd32533f4ce1b1f0f23d2c1cf6edf81a436ae7ebedbc3c16bf58763a058482e9
  Stored in directory: /tmp/pip-ephem-wheel-cache-_5_97e0x/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [9]:
from google.colab import files
import json
import whisper

# ============================================
# 1. Whisper 실행 (음성 인식)
# ============================================
print("🎙️ 음성 인식 시작...")
model = whisper.load_model("turbo")
transcription_result = model.transcribe(preprocessed_path, language="ko")

# ============================================
# 2. 세그먼트된 STT 결과 추출
# ============================================
print("\n📝 STT 세그먼트 추출 중...")

stt_segments = []
for seg in transcription_result['segments']:
    stt_segments.append({
        "text": seg['text'].strip(),
        "start": round(seg['start'], 1),
        "end": round(seg['end'], 1)
    })

# ============================================
# 3. STT 세그먼트 JSON 저장 및 다운로드
# ============================================
stt_output_filename = "stt_segments.json"
with open(stt_output_filename, 'w', encoding='utf-8') as f:
    json.dump(stt_segments, f, ensure_ascii=False, indent=2)

print(f"\n✅ STT 세그먼트 추출 완료: {len(stt_segments)}개")
print(f"📥 다운로드 시작: {stt_output_filename}")
files.download(stt_output_filename)

🎙️ 음성 인식 시작...


100%|██████████████████████████████████████| 1.51G/1.51G [00:06<00:00, 232MiB/s]



📝 STT 세그먼트 추출 중...

✅ STT 세그먼트 추출 완료: 1426개
📥 다운로드 시작: stt_segments.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 병합


In [10]:
# ============================================
# 4. Senko + Whisper 병합 (화자별 텍스트)
# ============================================
print("\n🔗 화자 분리 결과와 음성 인식 결과 병합 중...")

final_transcript = []
whisper_segments = transcription_result['segments']
whisper_idx = 0

# Senko의 merged_segments를 순회하며 텍스트 매칭
for seg in senko_result["merged_segments"]:
    speaker = seg['speaker']
    start_diarization = seg['start']
    end_diarization = seg['end']

    segment_text = ""

    # Whisper 세그먼트 탐색
    while whisper_idx < len(whisper_segments):
        ws = whisper_segments[whisper_idx]
        start_whisper = ws['start']
        end_whisper = ws['end']

        # Whisper 세그먼트가 현재 Diarization 세그먼트 내에 있는 경우
        if start_whisper >= start_diarization and start_whisper < end_diarization:
            segment_text += ws['text'].strip() + " "
            whisper_idx += 1
        elif start_whisper >= end_diarization:
            # 다음 Diarization 세그먼트로 이동
            break
        else:
            whisper_idx += 1

    # 텍스트가 있는 경우만 추가
    if segment_text:
        final_transcript.append({
            "speaker": speaker,
            "start": round(start_diarization, 2),
            "end": round(end_diarization, 2),
            "text": segment_text.strip()
        })

# ============================================
# 5. 결과 출력
# ============================================
print("\n## 🎤 최종 화자 분할 및 전사 결과 📝")
for entry in final_transcript:
    print(f"[{entry['speaker']}]: ({entry['start']:.2f}s - {entry['end']:.2f}s) {entry['text']}")

# ============================================
# 6. custom_result에 텍스트 추가
# ============================================
# STT 결과를 turns에 추가 (text 필드 추가)
for i, turn in enumerate(custom_result['turns']):
    # final_transcript에서 해당 구간 찾기
    matching_transcript = next(
        (t for t in final_transcript
         if t['speaker'] == turn['speaker_label']
         and t['start'] == turn['start']
         and t['end'] == turn['end']),
        None
    )
    if matching_transcript:
        turn['text'] = matching_transcript['text']
    else:
        turn['text'] = ""  # 텍스트가 없는 경우 빈 문자열

# ============================================
# 7. 최종 결과 JSON 저장 및 다운로드
# ============================================
final_output_filename = "diarization_with_stt_result.json"
with open(final_output_filename, 'w', encoding='utf-8') as f:
    json.dump(custom_result, f, ensure_ascii=False, indent=2)

print(f"\n📥 최종 결과 다운로드: {final_output_filename}")
files.download(final_output_filename)
print("✅ 완료!")


🔗 화자 분리 결과와 음성 인식 결과 병합 중...

## 🎤 최종 화자 분할 및 전사 결과 📝
[SPEAKER_01]: (4.57s - 16.30s) 좀 토킹 어바웃을 해볼까요? 여유롭게 처음에 아이트 브레이킹? 지금 누는 거예요? 네 저 재형님 말하세요 방금 강재림 20분 설치가 들어왔습니다
[SPEAKER_01]: (19.19s - 20.50s) 강재로요?
[SPEAKER_04]: (21.04s - 29.57s) 신분증 없다가 맞고 맞아가지고 여기 칼국수집 쪽 편의점 갔다가 거기서도 맞고 맞아가지고
[SPEAKER_01]: (29.57s - 31.50s) 왠지 안 올라왔잖아 반대쪽 편의점 가서 거기서 겨우 사왔어요
[SPEAKER_02]: (33.66s - 35.96s) 고등학생 같긴 해 그럴 리가 없는데?
[SPEAKER_04]: (49.58s - 51.03s) 아니면 중간중간에
[SPEAKER_01]: (51.03s - 62.47s) 그냥 계속 말을 해봐요 우리가 하는 것처럼 언제든지 치고 들어와 봐요 진행해 주시죠 아
[SPEAKER_02]: (62.54s - 74.73s) 너무 죄송해요 지금 이거 뭐지? 구글 구글 클라우드 SCT라는 걸 처음 저는 그냥 이게 이렇게 어려운 건 줄 몰라요 저 이렇게 어려운 줄 알 수 없잖아요 SM도예요 AST인데 개꿀
[SPEAKER_03]: (75.21s - 76.18s) 저는 이거 지금 구글 클라우드랑 구글 스토리지 맞나요?
[SPEAKER_02]: (76.18s - 120.70s) 이거 처음 써봐요 그래서 지금 약간 멘붕 왔는데 그리고 또 안 돼 심지어 약간 좀 어디 단계에서 막혔어요 그래가지고 약간 근데 이게 막혔다는 게 제 말은 LCT에서 막힌 게 아니라 구글 클라우드 허가 접근 권해에서 막혀서 지금 M1에서 막혀서 지금 좀 정체 상태고요 근데 해보고 싶긴 해요 그래서 오늘 집에 가서라도 해볼게요 네 근데 어제 지금 너무 저는 마음이 편한 게 다행히도 우리 팀원분들이 지금 어셈블리 AI 해보셨고 또 지금 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 완료!
